<center>
<h1 style="font-family:verdana">
 💻 🧑 Reconeixement d'entitats anomenades 🧑 💻


<p> 🎯 <b>Objectiu</b>: en aquesta segona part de la pràctica aprendrem a reconèixer entitats anomenades, és a dir, identificar i classificar entitats en una oració que poden ser (en el context de reserva de vols) la ciutat de sortida o d'arribada, la data del mes i el dia, si es de negocis o turista o altres categories específiques. Aquesta tasca se sol plantejar etiquetant cada paraula amb una etiqueta de la categoria de l'entitat a la qual correspon.

Tant la classificació d'intencions (tasca estudiada a la primera part) com el reconeixement d'entitats anomenades (Name Entity Recognition, NER) que veurem en aquesta part són components crucials dels sistemes de processament del llenguatge natural (Natural Language Processing, NLP) i sovint s'utilitzen junts per crear aplicacions d'intel·ligència artificial (IA) conversacionals més sofisticades. Per exemple, en un xatbot, la classificació d'intencions ajuda a entendre la intenció principal de l'usuari, mentre que el reconeixement d'entitats anomenades ajuda a extreure entitats rellevants per proporcionar respostes més contextualitzades.


<p> ✨ <b>Contingut</b>: els passos d'aquesta segona part seran similars a la primera. En primer lloc, analitzarem el conjunt de dades. En segon lloc, prepararem les dades per a poder entrenar el model. I finalment dissenyarem l'arquitectura i entrenarem el model. </p>


<p> ✏ <b>Exercicis</b>: en cada secció anireu trobant exercicis que haureu d'anar resolent. </p>

---

<h2> Índex </h2>

1. [Inspecció del conjunt de dades](#section-one)
  * [Exercici 1](#ex-one)
  * [Exercici 2](#ex-two)
2. [Preprocessament de dades](#section-two)
  * [Exercici 3](#ex-three)
  * [Exercici 4](#ex-four)
3. [Disseny del model i entrenament](#section-three)
  * [Exercici 5](#ex-five)
  * [Exercici 6](#ex-six)
4. [Lliurable](#section-four)

---

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report


<h1><a name="section-one"> 1. Inspecció del conjunt de dades </a></h1>

In [ ]:
!pip install gdown
!gdown "https://drive.google.com/uc?id=1u2wzXvsuscLeFHwXcDwMDaNDy0u_99-t"
!tar -zxf nlu_ATIS_data.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1u2wzXvsuscLeFHwXcDwMDaNDy0u_99-t
To: /content/nlu_ATIS_data.tar.gz
100% 122k/122k [00:00<00:00, 129MB/s]


In [ ]:
!ls data

test.csv  train.csv


En aquesta segona part de la pràctica utilitzarem el mateix dataset que a la primera part, però en aquest cas utilitzarem la **primera columna** que correspon a les **oracions** introduïdes pels usuaris i la **segona columna** on trobarem les oracions d'entrada en format **BILOU**.

El format BILOU  és un esquema d'etiquetatge que es fa servir a les tasques de reconeixement d'entitats anomenades. El nom *BILOU* representa les etiquetes utilitzades en aquest esquema:

*   **B** - Beginning: primer token d'una entitat.
*   **I** - Inside: token dins d'una entitat.
*   **L** - Last: últim token de l'entitat.
*   **O** - Outside: token que no pertany a cap entitat.
*   **U** - Unit: entitats d'un sol token

A continuació, carregarem les dades per visualitzar i poder entendre millor aquestes etiquetes.

---

 <h1><a name="ex-one"><center> ✏ Exercici 1 ✏</a></h1>

A continuació us demanem que carregueu els dos CSVs de la carpeta `data`: `train.csv`, `test.csv` utilitzant pandas. Recorda que aquests CSVs no tenen capçalera. Agafeu les 900 últimes lineas del fitxer `train.csv` per crear un dataframe per validació.

In [ ]:
#TODO
train_data = pd.read_csv('./data/train.csv', header=None)
val_data = train_data.tail(900)
train_data = pd.read_csv('./data/train.csv', header=None, nrows=4078)
test_data = pd.read_csv('./data/test.csv', header=None)

print('Training size:', len(train_data))
print('Validation dataset size:', len(val_data))
print('Test dataset size:', len(test_data))

Training size: 4078
Validation dataset size: 900
Test dataset size: 893


---

 <h1><a name="ex-two"><center> ✏ Exercici 2 ✏</a></h1>

Tal com hem introduït abans, per a aquest exercici ens centrarem en la **primera** i la **segona** columna. Per tant, ara us demanem que guardeu en les següents variables, les llistes corresponents a les oracions i a les etiquetes de les tres particions (`train`, `validation` i `test`).

In [ ]:
#TODO
train_sentences = list(train_data[0])
train_labels = list(train_data[1])

val_sentences = list(val_data[0])
val_labels = list(val_data[1])

test_sentences = list(test_data[0])
test_labels = list(test_data[1])


---

Si tot ha anat bé ja podem analitzar quin aspecte té el format BILOU.

In [ ]:
print(train_sentences[0])
print(train_labels[0])

i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
 "O O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time O O O B-toloc.city_name O B-arrive_time.time O O B-arrive_time.period_of_day"


Aquest hauria de ser el resultat obtingut si executeu la cel·la anterior:

```


i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
 "O O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time O O O B-toloc.city_name O B-arrive_time.time O O B-arrive_time.period_of_day"


```

En aquest exemple, `boston` i `838 am` corresponen a l'entitat "ciutat de sortida" i a l'entitat "hora de sortida" respectivament (`fromloc.city_name`,`depart_time.time`). `838` correspon al primer token que pertany la entitat `depart_time.time` i `am` al segon de la mateixa entitat. Els tokens com `i` o `want` no pertanyen a cap entitat.

In [ ]:
print(train_sentences[1])
print(train_labels[1])

what flights are available from pittsburgh to baltimore on thursday morning
 "O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day"


I aquest hauria de ser el resultat de la cel·la anterior:

```
what flights are available from pittsburgh to baltimore on thursday morning
 O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day
```

En l'anterior exemple, `what`, `flights`, `are`, `available`, `from`, `to` i `on` estan etiquetades com a no pertanyents a cap entitat. I, en canvi, `pittsburgh` pertany a l'entitat d'un sol token anomenada ciutat d'arribada (`toloc.city_name`). També hi ha entitats compostes com `baltimore` que pertany a l'entitat ciutat de sortida (`fromloc.city_name`) i `thursday morning` que pertanyen a les entitats data de sortida (`depart_date.day_name`, `depart_time.period_of_day`).

Vegem-ne la llista completa d'entitats.

In [ ]:
from collections import Counter

def count_unique_entities(list_of_label_sentences):
  flat_labels = []
  for labels in list_of_label_sentences:
    flat_labels += labels.split()
  c = Counter(flat_labels)
  return len(c), list(c.keys())

num_unique_entities, unique_entities = count_unique_entities(train_labels)

print("Number of different entities:", num_unique_entities)

print("Unique entities:", unique_entities)

Number of different entities: 216
Unique entities: ['"O', 'O', 'B-fromloc.city_name', 'B-depart_time.time', 'I-depart_time.time', 'B-toloc.city_name', 'B-arrive_time.time', 'B-arrive_time.period_of_day"', 'B-depart_date.day_name', 'B-depart_time.period_of_day"', 'B-flight_time', 'I-flight_time', 'I-fromloc.city_name', 'B-fromloc.city_name"', '"B-cost_relative', 'B-toloc.city_name"', '"B-round_trip', 'I-round_trip', 'B-cost_relative', 'B-fare_amount', 'I-fare_amount"', 'B-depart_date.today_relative', 'I-toloc.city_name', 'B-depart_date.day_name"', 'B-city_name"', 'I-toloc.city_name"', 'B-stoploc.city_name"', 'B-toloc.airport_code"', 'B-depart_time.time_relative', 'I-depart_time.time"', 'B-class_type', 'I-class_type', 'B-depart_date.date_relative', 'O"', 'B-airline_name', 'I-airline_name"', '"B-city_name', 'B-arrive_time.time_relative', 'B-arrive_time.time"', 'B-round_trip', 'I-airline_name', 'B-depart_time.start_time', 'I-depart_time.start_time', 'B-depart_time.end_time', 'I-depart_time

<h1><a name="section-two"> 2. Preprocessament de dades </a></h1>

El processament de les dades serà semblant al de la primera part d'aquesta pràctica.

---

 <h1><a name="ex-three"><center> ✏ Exercici 3 ✏</a></h1>

En aquest exercici us demanem que realitzeu els passos següents per preparar les dades.

 1. El primer pas serà construir el vocabulari a partir de les paraules presents a les oracions d'entrenament.

2. El segon pas serà convertir les oracions en seqüències de nombres enters usant el tokenitzador.

3. El tercer pas serà guardar la longitud original de cada oració. Això ens serà útil per evaluar el nostre model sense tenir en compte el padding.

3. Finalment, per aconseguir que totes les seqüències tinguen la mateixa longitud, fixarem la longitud segons la màxima trobada a l'entrenament i afegirem zeros a les oracions de menor longitud.

Recordeu que podeu consultar com fer-ho a la documentació de la llibrería:
* <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer"> Tokenizer </a>
* <a href="https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences"> Pad Sequences </a>




In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

# 1) Construir el vocabulari només amb les oracions de train
tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(train_sentences)

# 2) Convertir oracions a seqüències d'índexs
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences   = tokenizer.texts_to_sequences(val_sentences)
test_sequences  = tokenizer.texts_to_sequences(test_sentences)

# 3) Longitud original abans del padding
len_train_sequences = [len(seq) for seq in train_sequences]
len_val_sequences   = [len(seq) for seq in val_sequences]
len_test_sequences  = [len(seq) for seq in test_sequences]

# Longitud màxima a partir del train
max_sequence_length = max(len_train_sequences) if len_train_sequences else 0

# 4) Padding i truncat post a la longitud màxima
train_pad_sequences = pad_sequences(
    train_sequences, maxlen=max_sequence_length, padding='post', truncating='post', value=0
)
val_pad_sequences = pad_sequences(
    val_sequences, maxlen=max_sequence_length, padding='post', truncating='post', value=0
)
test_pad_sequences = pad_sequences(
    test_sequences, maxlen=max_sequence_length, padding='post', truncating='post', value=0
)

# Mida de vocabulari
vocab_size = len(tokenizer.word_index) + 1
print("Vocab size:", vocab_size)
print("Max sequence length (train):", max_sequence_length)


Vocab size: 831
Max sequence length (train): 46


In [ ]:
train_pad_sequences[0]

array([ 12,  69,   1,  38,   2,   9,  64, 415,  84,  17,  75,  16,  13,
        64, 493,  16,   4,  36,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=int32)


---



Tal com vam fer a la primera part, aquí també hem de convertir les diferents classes d'entitats en vectors one-hot.

---

 <h1><a name="ex-four"><center> ✏ Exercici 4 ✏</a></h1>

Per aconseguir-ho haureu de seguir els passos següents.

 1. En primer lloc, haureu d'esbrinar quantes etiquetes diferents hi ha. Podeu prendre com a exemple la funció `count_unique_entities` per fer-ho. Tingueu en compte que haureu de modificar la funció, ja que per exemple aquesta funció considera que `B-depart_time.time`, `I-depart_time.time`, `L-depart_time.time`, `U-depart_time.time` són la mateixa entitat. En aquest exercici, necessitarem comptar-les per separat. També l'entitat O ha de ser considerada com una classe.

 2. El segon pas serà codificar les diferents classes trobades en etiquetes numèriques. Tingueu en compte que cada paraula de l'oració té una etiqueta i, per tant, per a cada oració tindrem una llista d'etiquetes. El *padding* el codificarem amb l'etiqueta corresponent a **O** (outside). Per ajudar-vos hem preparat el processament per a les etiquetes d'entrenament, intenteu comprendre el que es fa i així repetir-ho per a la partició de validació i test.

 3. Finalment, haureu de convertir les diferents classes a vectors one-hot. Recordeu de nou que per a cada oració tindrem una llista de vectors one-hot.


 Podeu consultar els apartats de la documentació:
 * <a href=https://www.tensorflow.org/guide/keras/understanding_masking_and_padding> Masking and Padding </a>

 * <a href=https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical> To Categorical </a>



In [ ]:
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# 1) Comptar entitats úniques de TRAIN (BIO/BILOU) i afegir <pad> al principi
_TAG_RE = re.compile(r"^(?:[BILU]-)?[A-Za-z_]+(?:\.[A-Za-z_]+)*$|^O$")

def _clean_tag(t: str) -> str:
    """Elimina espais i cometes inicials/finals."""
    t = t.strip()
    return re.sub(r'^["\']+|["\']+$', '', t)

def _split_labels(maybe_str_or_list):
    """
    Accepta:
      - string amb etiquetes separades per espais: 'O B-foo I-foo'
      - llista d'etiquetes: ['O','B-foo','I-foo']
    Neteja i filtra etiquetes no vàlides.
    """
    if isinstance(maybe_str_or_list, str):
        raw = maybe_str_or_list.strip().split()
    else:
        raw = list(maybe_str_or_list)

    out = []
    for tok in raw:
        if not isinstance(tok, str) or not tok.strip():
            continue
        tag = _clean_tag(tok)
        if _TAG_RE.match(tag):
            out.append(tag)
    return out

def count_unique_entities_in_order(list_of_label_sentences):
    """
    Retorna:
      - nombre de classes + 1 (per <pad>)
      - llista de classes (sense <pad>) en ordre d'aparició a TRAIN
      - llista imprimible amb <pad> al principi
    """
    seen = set()
    classes = []
    for sent in list_of_label_sentences:
        for tag in _split_labels(sent):
            if tag not in seen:
                seen.add(tag)
                classes.append(tag)
    if 'O' not in seen:
        classes.append('O')
    printable = ['<pad>'] + classes
    return len(printable), classes, printable

# Variables existents:
#   train_labels
#   max_sequence_length
#   train_pad_sequences, val_pad_sequences, test_pad_sequences

num_unique_entities, classes, unique_entities_print = count_unique_entities_in_order(train_labels)
print("Number of different entities:", num_unique_entities)
print("Unique entities:", unique_entities_print)

# 2) LabelEncoder amb classes de TRAIN (sense '<pad>') + padding amb 'O'
label_encoder = LabelEncoder()
label_encoder.fit(classes)
o_ix = int(label_encoder.transform(['O'])[0])

def labels_to_ids(list_of_label_sentences):
    ids_per_sentence = []
    for sent in list_of_label_sentences:
        tags = _split_labels(sent)
        ids_per_sentence.append(label_encoder.transform(tags))
    return ids_per_sentence

def pad_or_truncate_label_ids(list_of_label_id_sents, max_len, pad_value):
    out = []
    for ids in list_of_label_id_sents:
        ids = list(ids)
        if len(ids) >= max_len:
            out.append(ids[:max_len])
        else:
            out.append(ids + [pad_value] * (max_len - len(ids)))
    return np.array(out, dtype=np.int32)

def remove_sentences(list_labels, list_sequences, allowed_set):
    """
    Elimina oracions amb etiquetes no vistes a TRAIN.
    Imprimeix 'labels to remove: [...]' amb duplicats i ordre d'aparició.
    """
    bad_idx = set()
    labels_to_remove = []

    for idx, raw in enumerate(list_labels):
        tags = _split_labels(raw)
        unknowns = [t for t in tags if t not in allowed_set]
        if unknowns:
            bad_idx.add(idx)
            labels_to_remove.extend(unknowns)

    kept_labels    = [l for i, l in enumerate(list_labels)    if i not in bad_idx]
    kept_sequences = [s for i, s in enumerate(list_sequences) if i not in bad_idx]

    if labels_to_remove:
        print("labels to remove:  ", labels_to_remove)

    return kept_labels, np.array(kept_sequences)

# TRAIN
train_numerical_labels = labels_to_ids(train_labels)
train_pad_labels      = pad_or_truncate_label_ids(train_numerical_labels, max_sequence_length, o_ix)

# VALIDATION
_val_allowed = set(classes)
val_labels,  val_pad_sequences  = remove_sentences(val_labels,  val_pad_sequences,  _val_allowed)
val_numerical_labels  = labels_to_ids(val_labels)
val_pad_labels        = pad_or_truncate_label_ids(val_numerical_labels,  max_sequence_length, o_ix)

# TEST
_test_allowed = set(classes)
test_labels, test_pad_sequences = remove_sentences(test_labels, test_pad_sequences, _test_allowed)
test_numerical_labels = labels_to_ids(test_labels)
test_pad_labels       = pad_or_truncate_label_ids(test_numerical_labels, max_sequence_length, o_ix)

# 3) One-hot per token. num_classes = len(classes) (sense '<pad>')
num_classes = len(label_encoder.classes_)
train_labels_one_hot = to_categorical(train_pad_labels, num_classes=num_classes)
val_labels_one_hot   = to_categorical(val_pad_labels,   num_classes=num_classes)
test_labels_one_hot  = to_categorical(test_pad_labels,  num_classes=num_classes)

try:
    assert train_pad_sequences.shape[1] == train_pad_labels.shape[1] == max_sequence_length
    assert val_pad_sequences.shape[1]   == val_pad_labels.shape[1]   == max_sequence_length
    assert test_pad_sequences.shape[1]  == test_pad_labels.shape[1]  == max_sequence_length
except NameError:
    pass

print("Shapes one-hot ->",
      "train:", train_labels_one_hot.shape,
      "val:",   val_labels_one_hot.shape,
      "test:",  test_labels_one_hot.shape)


Number of different entities: 120
Unique entities: ['<pad>', 'O', 'B-fromloc.city_name', 'B-depart_time.time', 'I-depart_time.time', 'B-toloc.city_name', 'B-arrive_time.time', 'B-arrive_time.period_of_day', 'B-depart_date.day_name', 'B-depart_time.period_of_day', 'B-flight_time', 'I-flight_time', 'I-fromloc.city_name', 'B-cost_relative', 'B-round_trip', 'I-round_trip', 'B-fare_amount', 'I-fare_amount', 'B-depart_date.today_relative', 'I-toloc.city_name', 'B-city_name', 'B-stoploc.city_name', 'B-toloc.airport_code', 'B-depart_time.time_relative', 'B-class_type', 'I-class_type', 'B-depart_date.date_relative', 'B-airline_name', 'I-airline_name', 'B-arrive_time.time_relative', 'B-depart_time.start_time', 'I-depart_time.start_time', 'B-depart_time.end_time', 'I-depart_time.end_time', 'B-fromloc.airport_name', 'I-fromloc.airport_name', 'B-toloc.state_name', 'B-depart_date.day_number', 'I-depart_date.day_number', 'B-depart_date.month_name', 'B-mod', 'B-fare_basis_code', 'B-transport_type', 'B

<h1><a name="section-three"> 3. Disseny del model i entrenament </a></h1>

---
<h1><a name="ex-five"><center> ✏ Exercici 5 ✏</a></h1>

De forma similar com féreu a la primera part de la pràctica us demanem que dissenyeu l'arquitectura i entreneu el model. Podeu fer servir una arquitectura similar. Ara bé, per capturar les dependències seqüencials podeu canviar la capa de GlobalMaxPooling1D per una LSTM.

In [ ]:
s = train_pad_labels.shape
train_flat_labels = train_pad_labels.reshape(s[0]*s[1])
train_unq_labels = np.unique(train_flat_labels)
print(train_flat_labels.shape, train_unq_labels)

(187588,) [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118]


In [ ]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, inputs):
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        position_embeddings = self.pos_emb(positions)
        token_embeddings = self.token_emb(inputs)
        return token_embeddings + position_embeddings





In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential

# 1) Afegeix classe <pad> a les etiquetes (pad detectat pel 0 a les seqüències d'entrada)
old_num_classes = int(train_labels_one_hot.shape[-1])
PAD_IX = old_num_classes
num_classes = old_num_classes + 1

def inject_pad_class(y_ids, x_pad_sequences, pad_ix):
    """Substitueix posicions de padding (tokens == 0) per la classe PAD a y."""
    y = y_ids.copy()
    mask = (x_pad_sequences == 0)
    y[mask] = pad_ix
    return y

train_y_ids_ext = inject_pad_class(train_pad_labels, train_pad_sequences, PAD_IX)
val_y_ids_ext   = inject_pad_class(val_pad_labels,   val_pad_sequences,   PAD_IX)
test_y_ids_ext  = inject_pad_class(test_pad_labels,  test_pad_sequences,  PAD_IX)

# One-hot nou amb 120 classes (inclou <pad>)
train_labels_one_hot = to_categorical(train_y_ids_ext, num_classes=num_classes)
val_labels_one_hot   = to_categorical(val_y_ids_ext,   num_classes=num_classes)
test_labels_one_hot  = to_categorical(test_y_ids_ext,  num_classes=num_classes)

# Pesos per ignorar <pad> a la loss
train_w = (train_y_ids_ext != PAD_IX).astype("float32")
val_w   = (val_y_ids_ext   != PAD_IX).astype("float32")
test_w  = (test_y_ids_ext  != PAD_IX).astype("float32")

# 2) Model (sortida de 120)
embedding_dim = 128
vocab_size = int(np.max(train_pad_sequences)) + 1

model = Sequential([
    keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        mask_zero=True,
        input_length=train_pad_sequences.shape[1]
    ),
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(num_classes, activation="softmax"))
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# 3) Entrenament (sample_weight per no penalitzar <pad>)
batch_size = 32
epochs = 10
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2)
]

history = model.fit(
    x=train_pad_sequences,
    y=train_labels_one_hot,
    sample_weight=train_w,
    validation_data=(val_pad_sequences, val_labels_one_hot, val_w),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

# 4) Avaluació i predicció
loss, accuracy = model.evaluate(
    test_pad_sequences, test_labels_one_hot, sample_weight=test_w, batch_size=batch_size, verbose=1
)
print(f"Test accuracy: {accuracy:.2f}")

preds = model.predict(test_pad_sequences, batch_size=batch_size, verbose=0)
print("preds.shape ->", preds.shape)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 12s 21ms/step - accuracy: 0.1546 - loss: 2.9033 - val_accuracy: 0.1913 - val_loss: 1.0332 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.2028 - loss: 0.8483 - val_accuracy: 0.2067 - val_loss: 0.7214 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.2185 - loss: 0.5634 - val_accuracy: 0.2200 - val_loss: 0.5324 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.2262 - loss: 0.3725 - val_accuracy: 0.2255 - val_loss: 0.4186 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2336 - loss: 0.2631 - val_accuracy: 0.2283 - val_loss: 0.3594 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2366 - loss: 0.1946 - val_accuracy: 0.2302 - val_loss: 0.3227 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2425 - loss: 0

In [ ]:
preds = model.predict(test_pad_sequences)

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [ ]:
preds.shape

(885, 46, 120)

In [ ]:
def preds_to_index(preds, seq_lens):
  '''
  Turn predictions to numerical indexes, flatten the sentences and discard padding.
  '''
  idx_preds = []
  for pred, seq_len in zip(preds,seq_lens):
      for l in range(seq_len):
        idx_preds.append(np.argmax(pred[l]))
  return idx_preds



Com ja sabeu, NER es una tasca on les dades estan molt desbalancejades. La gran majoria de les nostres etiquetes serán 'O' (outside). Es pot donar el cas on el model tingui una accuracy molt alta predint sempre 'O'.

Per mesurar millor com de bó és el nostre model, calcularem la F1 score per cada classe, així com la mitjana (macro average). Quan prepareu el vostre document, heu de reportar aquesta mitjana com a mètrica del vostre model, no l'acuracy.

In [ ]:
test_labels_idx = preds_to_index(test_labels_one_hot,len_test_sequences)
preds_idx = preds_to_index(preds, len_test_sequences)

print(classification_report(test_labels_idx, preds_idx, zero_division=1.0))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        20
           1       0.87      0.50      0.63        26
           2       0.86      0.95      0.90        79
           3       1.00      0.20      0.33         5
           4       0.29      0.11      0.16        18
           5       1.00      1.00      1.00         1
           6       0.70      1.00      0.82         7
           7       1.00      0.33      0.50         3
           8       0.67      0.67      0.67         3
          12       0.50      1.00      0.67         2
          13       1.00      1.00      1.00         3
          14       0.79      0.92      0.85        12
          15       0.71      0.86      0.77        14
          16       0.63      0.54      0.58        41
          17       0.96      0.96      0.96        24
          18       1.00      0.50      0.67         6
          19       1.00      1.00      1.00        33
          20       1.00    

A continuació podeu veure algunes prediccions del model.

In [ ]:
probs = model.predict(test_pad_sequences)
_predicted_labels = np.argmax(probs, axis=2)

predicted_labels = [list(label_encoder.inverse_transform(label)) for label in _predicted_labels]

for i in range(0, 10):
    print('Sentence: ', test_sentences[i])
    print('Original label: ', test_labels[i])
    print('Predicted label: ', predicted_labels[i])
    print()

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Sentence:  i would like to find a flight from charlotte to las vegas that makes a stop in st. louis
Original label:   "O O O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O O O O O B-stoploc.city_name I-stoploc.city_name"
Predicted label:  [np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('B-fromloc.city_name'), np.str_('O'), np.str_('B-toloc.city_name'), np.str_('I-toloc.city_name'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('O'), np.str_('B-stoploc.city_name'), np.str_('I-stoploc.city_name'), np.str_('B-depart_time.time_relative'), np.str_('B-depart_time.time_relative'), np.str_('B-depart_time.time_relative'), np.str_('B-depart_time.time_relative'), np.str_('B-depart_time.time_relative'), np.str_('B-depart_time.time_relative'), np.str_('B-depart_time.time_relative'), np.str_('B-depart_time.time_relative'), np.str_('B-depart_time.tim

---

 <h1><a name="ex-six"><center> ✏ Exercici 6 ✏ </a></h1>

Modifiqueu els següents paràmetres del model anterior i analitzeu com afecten a la seva *accuracy*:

 1. **Mida dels Embeddings.** Proveu diferents mides d'*Embeddings* i observeu com canvia l'*accuracy* del model. Heu d'explicar les vostres conclusions.

 2. **Xarxes Convolucionals.** Afegiu capes convolucionals al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació a l'hora d'escollir-los. Recordeu, que també podeu provar diferents configuracions de *pooling*.

 3. **Xarxes Recurrents.**  Afegiu capes recurrents al vostre model (LSTM, GRU). Expliqueu amb detall els valors que heu provat i la vostra motivació.

 4. ** Transformer** Afegiu blocs de Transformer al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació.

 5. **Regularització.** Quan proveu configuracions amb més paràmetres veureu que el model comença a tenir *overfitting* molt prompte durant l'entrenament. Afegiu *Dropout* al vostre model. Heu d'explicar la vostra decisió de valors i de posició dins de la xarxa.


 6. **Balancejat de les classes.** Si analitzeu el dataset, veureu que la freqüència de les classes està molt desbalancejada. Keras us permet afegir un pes per a cada classe a l'hora de calcular la loss (Mireu el paràmetre "class_weigth" a la documentació https://keras.io/api/models/model_training_apis/). Calculeu un pes per a cada classe i afegiu-lo al mètode fit del vostre model.

 ---

 1. **Mida dels Embeddings.** Proveu diferents mides d'*Embeddings* i observeu com canvia l'*accuracy* del model. Heu d'explicar les vostres conclusions.

In [ ]:
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# Helpers
def preds_to_index(preds, seq_lens):
    """Converteix probabilitats en índexs (argmax) i descarta padding segons la longitud real."""
    idx_preds = []
    for pred, seq_len in zip(preds, seq_lens):
        for l in range(seq_len):
            idx_preds.append(int(np.argmax(pred[l])))
    return idx_preds

def final_three_metrics(y_true_idx, y_pred_idx):
    """Retorna accuracy, macro(avg) i weighted(avg)."""
    acc = accuracy_score(y_true_idx, y_pred_idx)
    mP, mR, mF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="macro", zero_division=1.0
    )
    wP, wR, wF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="weighted", zero_division=1.0
    )
    return {
        "accuracy": float(acc),
        "macro_precision": float(mP),
        "macro_recall": float(mR),
        "macro_f1": float(mF1),
        "weighted_precision": float(wP),
        "weighted_recall": float(wR),
        "weighted_f1": float(wF1),
        "support_total": int(len(y_true_idx)),
    }

def run_embeddings_and_print_table(
    embed_dims=(32, 64, 128, 256, 512, 1024, 2048, 4096),
    lstm_units=128,
    batch_size=32,
    epochs=10,
    print_epoch_logs=True
):
    """Entrena per a cada embedding_dim i imprimeix una taula amb mètriques finals."""
    results = []
    resumen_por_emb = {}

    vocab_size = int(np.max(train_pad_sequences)) + 1
    max_len    = int(train_pad_sequences.shape[1])
    num_classes = int(train_labels_one_hot.shape[-1])

    for emb in embed_dims:
        print("\n" + "="*90)
        print(f"Entrenando con embedding_dim = {emb}")
        print("="*90)
        tf.keras.backend.clear_session()
        start = time.time()

        model = Sequential([
            keras.layers.Embedding(
                input_dim=vocab_size,
                output_dim=emb,
                mask_zero=True,
                input_length=max_len
            ),
            keras.layers.LSTM(lstm_units, return_sequences=True),
            keras.layers.TimeDistributed(
                keras.layers.Dense(num_classes, activation="softmax")
            ),
        ])

        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

        callbacks = [
            keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2),
        ]

        history = model.fit(
            x=train_pad_sequences,
            y=train_labels_one_hot,
            sample_weight=train_w,  # ignora <pad> a la loss
            validation_data=(val_pad_sequences, val_labels_one_hot, val_w),
            batch_size=batch_size,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1 if print_epoch_logs else 0
        )

        preds = model.predict(test_pad_sequences, batch_size=batch_size, verbose=0)
        y_true_idx = preds_to_index(test_labels_one_hot, len_test_sequences)
        y_pred_idx = preds_to_index(preds,                len_test_sequences)

        final3 = final_three_metrics(y_true_idx, y_pred_idx)
        resumen_por_emb[emb] = final3

        results.append({
            "embedding_dim": emb,
            "accuracy": final3["accuracy"],
            "macro_precision": final3["macro_precision"],
            "macro_recall": final3["macro_recall"],
            "macro_f1": final3["macro_f1"],
            "weighted_precision": final3["weighted_precision"],
            "weighted_recall": final3["weighted_recall"],
            "weighted_f1": final3["weighted_f1"],
            "support_total": final3["support_total"],
        })

    df = pd.DataFrame(results).sort_values("embedding_dim")
    cols = [
        "embedding_dim",
        "accuracy",
        "macro_precision", "macro_recall", "macro_f1",
        "weighted_precision", "weighted_recall", "weighted_f1",
        "support_total"
    ]
    df = df[cols]
    print("\nResumen por tamaño de embedding (los tres del final):")
    print(df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

    return df, resumen_por_emb

tabla_final, resumen_por_emb = run_embeddings_and_print_table(
    embed_dims=(32, 64, 128, 256, 512, 1024, 2048, 4096),
    lstm_units=128,
    batch_size=32,
    epochs=10,
    print_epoch_logs=True
)



Entrenando con embedding_dim = 32


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.1496 - loss: 3.3110 - val_accuracy: 0.1776 - val_loss: 1.3482 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.1901 - loss: 1.1855 - val_accuracy: 0.1906 - val_loss: 1.0178 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.1988 - loss: 0.8951 - val_accuracy: 0.1974 - val_loss: 0.8794 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2061 - loss: 0.7396 - val_accuracy: 0.2032 - val_loss: 0.7772 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2118 - loss: 0.6164 - val_accuracy: 0.2074 - val_loss: 0.7010 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.2151 - loss: 0.5345 - val_accuracy: 0.2119 - val_loss: 0.6408 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.2183 - loss: 0

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.1480 - loss: 3.1516 - val_accuracy: 0.1807 - val_loss: 1.2040 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.1914 - loss: 1.0398 - val_accuracy: 0.1956 - val_loss: 0.8982 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.2061 - loss: 0.7464 - val_accuracy: 0.2058 - val_loss: 0.7276 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2131 - loss: 0.5666 - val_accuracy: 0.2146 - val_loss: 0.5992 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2257 - loss: 0.4314 - val_accuracy: 0.2216 - val_loss: 0.4994 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2277 - loss: 0.3328 - val_accuracy: 0.2251 - val_loss: 0.4338 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.2331 - loss: 0.

...

Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.1662 - loss: 2.3016 - val_accuracy: 0.2141 - val_loss: 0.6574 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.2270 - loss: 0.4687 - val_accuracy: 0.2272 - val_loss: 0.3689 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2354 - loss: 0.2255 - val_accuracy: 0.2310 - val_loss: 0.2860 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2394 - loss: 0.1314 - val_accuracy: 0.2324 - val_loss: 0.2519 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2437 - loss: 0.0903 - val_accuracy: 0.2336 - val_loss: 0.2358 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2473 - loss: 0.0651 - val_accuracy: 0.2348 - val_loss: 0.2268 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2437 - loss: 0.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.1715 - loss: 2.0831 - val_accuracy: 0.2202 - val_loss: 0.5364 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2306 - loss: 0.3648 - val_accuracy: 0.2294 - val_loss: 0.3204 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2383 - loss: 0.1744 - val_accuracy: 0.2325 - val_loss: 0.2599 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2421 - loss: 0.1081 - val_accuracy: 0.2336 - val_loss: 0.2376 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2447 - loss: 0.0688 - val_accuracy: 0.2342 - val_loss: 0.2276 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2461 - loss: 0.0502 - val_accuracy: 0.2347 - val_loss: 0.2219 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.2456 - loss: 0.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.1739 - loss: 1.8853 - val_accuracy: 0.2226 - val_loss: 0.4369 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2358 - loss: 0.2815 - val_accuracy: 0.2309 - val_loss: 0.2817 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2392 - loss: 0.1396 - val_accuracy: 0.2329 - val_loss: 0.2373 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2421 - loss: 0.0851 - val_accuracy: 0.2332 - val_loss: 0.2312 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.2423 - loss: 0.0637 - val_accuracy: 0.2344 - val_loss: 0.2165 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.2445 - loss: 0.0475 - val_accuracy: 0.2347 - val_loss: 0.2166 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2456 - loss: 0.

##Xarxes Convolucionals.
Afegiu capes convolucionals al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació a l'hora d'escollir-los. Recordeu, que també podeu provar diferents configuracions de pooling.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Embedding, Conv1D, Dropout, MaxPooling1D,
                                     AveragePooling1D, UpSampling1D, Cropping1D,
                                     Concatenate)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# -----------------------------
# Utilitats
# -----------------------------
def preds_to_index(preds, seq_lens):
    """Converteix probabilitats en índexs (argmax) i descarta el padding segons la longitud real."""
    idx_preds = []
    for pred, seq_len in zip(preds, seq_lens):
        for l in range(seq_len):
            idx_preds.append(int(np.argmax(pred[l])))
    return idx_preds

def final_three_metrics(y_true_idx, y_pred_idx):
    """Retorna accuracy, macro(avg) i weighted(avg)."""
    acc = accuracy_score(y_true_idx, y_pred_idx)
    mP, mR, mF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="macro", zero_division=1.0
    )
    wP, wR, wF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="weighted", zero_division=1.0
    )
    return {
        "accuracy": float(acc),
        "macro_precision": float(mP),
        "macro_recall": float(mR),
        "macro_f1": float(mF1),
        "weighted_precision": float(wP),
        "weighted_recall": float(wR),
        "weighted_f1": float(wF1),
        "support_total": int(len(y_true_idx)),
    }

# -----------------------------
# Construcció de models CNN per token
# -----------------------------
embedding_dim = 256
vocab_size = int(np.max(train_pad_sequences)) + 1
max_len = int(train_pad_sequences.shape[1])
num_classes = int(train_labels_one_hot.shape[-1])

def build_conv_token(kernel_size=5, filters=128, dropout=0.2):
    """Conv1D per token (sense pooling), manté la longitud i dona sortida per timestep."""
    inp = keras.Input(shape=(max_len,), dtype="int32")
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  input_length=max_len, mask_zero=False)(inp)
    x = Conv1D(filters=filters, kernel_size=kernel_size,
               padding="same", activation="relu")(x)
    x = Dropout(dropout)(x)
    out = Conv1D(filters=num_classes, kernel_size=1,
                 padding="same", activation="softmax")(x)
    model = keras.Model(inp, out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_conv_pool_token(kernel_size=5, filters=128, pool_type="max",
                          pool_size=2, dropout=0.2):
    """Conv1D + pooling + UpSampling + Cropping per mantenir la longitud."""
    inp = keras.Input(shape=(max_len,), dtype="int32")
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  input_length=max_len, mask_zero=False)(inp)
    x = Conv1D(filters=filters, kernel_size=kernel_size,
               padding="same", activation="relu")(x)
    if pool_type == "max":
        x = MaxPooling1D(pool_size=pool_size, padding="same")(x)
    elif pool_type == "avg":
        x = AveragePooling1D(pool_size=pool_size, padding="same")(x)
    else:
        raise ValueError("pool_type ha de ser 'max' o 'avg'")
    x = UpSampling1D(size=pool_size)(x)
    extra = (pool_size - (max_len % pool_size)) % pool_size
    if extra > 0:
        x = Cropping1D(cropping=(0, extra))(x)
    x = Dropout(dropout)(x)
    out = Conv1D(filters=num_classes, kernel_size=1,
                 padding="same", activation="softmax")(x)
    model = keras.Model(inp, out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_textcnn_token(kernels=(3, 4, 5), filters=96, dropout=0.2):
    """TextCNN per token amb branques paral·leles i concatenació de canals."""
    inp = keras.Input(shape=(max_len,), dtype="int32")
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  input_length=max_len, mask_zero=False)(inp)
    branches = []
    for k in kernels:
        c = Conv1D(filters=filters, kernel_size=k, padding="same", activation="relu")(x)
        branches.append(c)
    if len(branches) > 1:
        x = Concatenate(axis=-1)(branches)
    else:
        x = branches[0]
    x = Dropout(dropout)(x)
    out = Conv1D(filters=num_classes, kernel_size=1,
                 padding="same", activation="softmax")(x)
    model = keras.Model(inp, out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# -----------------------------
# Experiments CNN
# -----------------------------
experiments = [
    ("conv_k3_nopool",        build_conv_token(kernel_size=3, filters=128)),
    ("conv_k5_nopool",        build_conv_token(kernel_size=5, filters=128)),
    ("conv_k7_nopool",        build_conv_token(kernel_size=7, filters=128)),
    ("conv_k5_maxpool_up2",   build_conv_pool_token(kernel_size=5, filters=128, pool_type="max", pool_size=2)),
    ("conv_k5_avgpool_up2",   build_conv_pool_token(kernel_size=5, filters=128, pool_type="avg", pool_size=2)),
    ("textcnn_k345",          build_textcnn_token(kernels=(3, 4, 5), filters=64)),
]

# -----------------------------
# Entrenament i taula final
# -----------------------------
batch_size = 32
epochs = 10
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2),
]

rows = []
for name, model in experiments:
    print("\n" + "="*90)
    print(f"Entrenando {name}  (embedding_dim=512)")
    print("="*90)
    hist = model.fit(
        x=train_pad_sequences,
        y=train_labels_one_hot,
        sample_weight=train_w,  # ignora <pad>
        validation_data=(val_pad_sequences, val_labels_one_hot, val_w),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )

    preds = model.predict(test_pad_sequences, batch_size=batch_size, verbose=0)
    y_true_idx = preds_to_index(test_labels_one_hot, len_test_sequences)
    y_pred_idx = preds_to_index(preds,                len_test_sequences)
    met = final_three_metrics(y_true_idx, y_pred_idx)

    rows.append({
        "model": name,
        "accuracy": met["accuracy"],
        "macro_precision": met["macro_precision"],
        "macro_recall": met["macro_recall"],
        "macro_f1": met["macro_f1"],
        "weighted_precision": met["weighted_precision"],
        "weighted_recall": met["weighted_recall"],
        "weighted_f1": met["weighted_f1"],
        "support_total": met["support_total"],
    })

df = pd.DataFrame(rows)
df = df[[
    "model",
    "accuracy",
    "macro_precision", "macro_recall", "macro_f1",
    "weighted_precision", "weighted_recall", "weighted_f1",
    "support_total"
]].sort_values("model")

print("\nResumen CNN (embedding=256):")
print(df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Entrenando conv_k3_nopool  (embedding_dim=512)
Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.1744 - loss: 0.6389 - val_accuracy: 0.2198 - val_loss: 0.1326 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2285 - loss: 0.0879 - val_accuracy: 0.2269 - val_loss: 0.0975 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2342 - loss: 0.0520 - val_accuracy: 0.2290 - val_loss: 0.0889 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2370 - loss: 0.0372 - val_accuracy: 0.2300 - val_loss: 0.0855 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2381 - loss: 0.0304 - val_accuracy: 0.2307 - val_loss: 0.0839 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2390 - loss: 0.0263 - val_accuracy: 0.2311 - val_loss: 0.0842 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━

##Xarxes Recurrents.
Afegiu capes recurrents al vostre model (LSTM, GRU). Expliqueu amb detall els valors que heu provat i la vostra motivació.     

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, TimeDistributed, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# ------------------------------------------------------------------
# Funcions auxiliars
# ------------------------------------------------------------------
def preds_to_index(preds, seq_lens):
    """Converteix probabilitats en índexs (argmax) i descarta el padding segons la longitud real."""
    idx_preds = []
    for pred, seq_len in zip(preds, seq_lens):
        for l in range(int(seq_len)):
            idx_preds.append(int(np.argmax(pred[l])))
    return idx_preds

def final_three_metrics(y_true_idx, y_pred_idx):
    """Retorna accuracy, macro(avg) i weighted(avg)."""
    acc = accuracy_score(y_true_idx, y_pred_idx)
    mP, mR, mF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="macro", zero_division=1.0
    )
    wP, wR, wF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="weighted", zero_division=1.0
    )
    return {
        "accuracy": float(acc),
        "macro_precision": float(mP),
        "macro_recall": float(mR),
        "macro_f1": float(mF1),
        "weighted_precision": float(wP),
        "weighted_recall": float(wR),
        "weighted_f1": float(wF1),
        "support_total": int(len(y_true_idx)),
    }

# Si no hi ha len_test_sequences, es calcula a partir de les entrades
try:
    len_test_sequences
except NameError:
    len_test_sequences = (test_pad_sequences != 0).sum(axis=1)

# ------------------------------------------------------------------
# Paràmetres compartits
# ------------------------------------------------------------------
embedding_dim = 256
vocab_size    = int(np.max(train_pad_sequences)) + 1
max_len       = int(train_pad_sequences.shape[1])
num_classes   = int(train_labels_one_hot.shape[-1])

batch_size = 32
epochs     = 10
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2),
]

# ------------------------------------------------------------------
# Constructors RNN per token (sortida [B, T, C])
# ------------------------------------------------------------------
def build_rnn_token(cell="lstm", units=128, bidirectional=False, dropout=0.2, rec_dropout=0.0):
    """
    Embedding(mask_zero=True) -> (Bi)LSTM/GRU return_sequences=True -> TimeDistributed(Dense softmax).
    """
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                        input_length=max_len, mask_zero=True))
    rnn_layer = LSTM(units, return_sequences=True, dropout=dropout, recurrent_dropout=rec_dropout) \
                if cell.lower() == "lstm" else \
                GRU(units,  return_sequences=True, dropout=dropout, recurrent_dropout=rec_dropout)

    if bidirectional:
        model.add(Bidirectional(rnn_layer))
    else:
        model.add(rnn_layer)

    model.add(TimeDistributed(Dense(num_classes, activation="softmax")))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# ------------------------------------------------------------------
# Experiments (LSTM / GRU, uni i bidireccional, 64 i 128)
# ------------------------------------------------------------------
experiments = [
    ("lstm_64",    build_rnn_token(cell="lstm", units=64,  bidirectional=False)),
    ("lstm_128",   build_rnn_token(cell="lstm", units=128, bidirectional=False)),
    ("bilstm_64",  build_rnn_token(cell="lstm", units=64,  bidirectional=True)),
    ("bilstm_128", build_rnn_token(cell="lstm", units=128, bidirectional=True)),
    ("gru_64",     build_rnn_token(cell="gru",  units=64,  bidirectional=False)),
    ("gru_128",    build_rnn_token(cell="gru",  units=128, bidirectional=False)),
    ("bigru_64",   build_rnn_token(cell="gru",  units=64,  bidirectional=True)),
    ("bigru_128",  build_rnn_token(cell="gru",  units=128, bidirectional=True)),
]

# ------------------------------------------------------------------
# Entrenament, avaluació i taula de resultats
# ------------------------------------------------------------------
rows = []
for name, model in experiments:
    print("\n" + "="*90)
    print(f"Entrenando {name}  (embedding_dim=256)")
    print("="*90)

    model.fit(
        x=train_pad_sequences,
        y=train_labels_one_hot,
        sample_weight=train_w,
        validation_data=(val_pad_sequences, val_labels_one_hot, val_w),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )

    preds = model.predict(test_pad_sequences, batch_size=batch_size, verbose=0)
    y_true_idx = preds_to_index(test_labels_one_hot, len_test_sequences)
    y_pred_idx = preds_to_index(preds,                len_test_sequences)

    met = final_three_metrics(y_true_idx, y_pred_idx)
    rows.append({
        "model": name,
        "accuracy": met["accuracy"],
        "macro_precision": met["macro_precision"],
        "macro_recall": met["macro_recall"],
        "macro_f1": met["macro_f1"],
        "weighted_precision": met["weighted_precision"],
        "weighted_recall": met["weighted_recall"],
        "weighted_f1": met["weighted_f1"],
        "support_total": met["support_total"],
    })

df = pd.DataFrame(rows)[[
    "model",
    "accuracy",
    "macro_precision", "macro_recall", "macro_f1",
    "weighted_precision", "weighted_recall", "weighted_f1",
    "support_total"
]].sort_values("model")

print("\nResumen RNN per-token (embedding=256):")
print(df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Entrenando lstm_64  (embedding_dim=256)
Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.1470 - loss: 3.0097 - val_accuracy: 0.1818 - val_loss: 1.2097 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.1904 - loss: 1.0855 - val_accuracy: 0.1956 - val_loss: 0.9106 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2037 - loss: 0.7902 - val_accuracy: 0.2087 - val_loss: 0.6872 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2192 - loss: 0.5607 - val_accuracy: 0.2203 - val_loss: 0.5229 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2273 - loss: 0.4006 - val_accuracy: 0.2218 - val_loss: 0.4377 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2294 - loss: 0.3133 - val_accuracy: 0.2236 - val_loss: 0.3898 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 

Comparando con el mejor modelo de CNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, TimeDistributed, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# ------------------------------------------------------------------
# Funcions auxiliars
# ------------------------------------------------------------------
def preds_to_index(preds, seq_lens):
    """Converteix probabilitats en índexs (argmax) i descarta el padding segons la longitud real."""
    idx_preds = []
    for pred, seq_len in zip(preds, seq_lens):
        for l in range(int(seq_len)):
            idx_preds.append(int(np.argmax(pred[l])))
    return idx_preds

def final_three_metrics(y_true_idx, y_pred_idx):
    """Retorna accuracy, macro(avg) i weighted(avg)."""
    acc = accuracy_score(y_true_idx, y_pred_idx)
    mP, mR, mF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="macro", zero_division=1.0
    )
    wP, wR, wF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="weighted", zero_division=1.0
    )
    return {
        "accuracy": float(acc),
        "macro_precision": float(mP),
        "macro_recall": float(mR),
        "macro_f1": float(mF1),
        "weighted_precision": float(wP),
        "weighted_recall": float(wR),
        "weighted_f1": float(wF1),
        "support_total": int(len(y_true_idx)),
    }

# Si no hi ha len_test_sequences, es calcula a partir de les entrades
try:
    len_test_sequences
except NameError:
    len_test_sequences = (test_pad_sequences != 0).sum(axis=1)

# ------------------------------------------------------------------
# Paràmetres compartits
# ------------------------------------------------------------------
embedding_dim = 256
vocab_size    = int(np.max(train_pad_sequences)) + 1
max_len       = int(train_pad_sequences.shape[1])
num_classes   = int(train_labels_one_hot.shape[-1])

batch_size = 32
epochs     = 10
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2),
]

# ------------------------------------------------------------------
# Constructors RNN per token (sortida [B, T, C])
# ------------------------------------------------------------------
def build_rnn_token(cell="lstm", units=128, bidirectional=False, dropout=0.2, rec_dropout=0.0):
    """
    Embedding(mask_zero=True) -> (Bi)LSTM/GRU return_sequences=True -> TimeDistributed(Dense softmax).
    """
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                        input_length=max_len, mask_zero=True))
    rnn_layer = LSTM(units, return_sequences=True, dropout=dropout, recurrent_dropout=rec_dropout) \
                if cell.lower() == "lstm" else \
                GRU(units,  return_sequences=True, dropout=dropout, recurrent_dropout=rec_dropout)

    if bidirectional:
        model.add(Bidirectional(rnn_layer))
    else:
        model.add(rnn_layer)

    # Projecció per token a num_classes
    model.add(TimeDistributed(Dense(num_classes, activation="softmax")))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

from tensorflow.keras.layers import Conv1D

def build_conv_k7_nopool():
    """CNN per token amb kernel 7 i sense pooling."""
    inp = keras.Input(shape=(max_len,), dtype="int32")
    x = Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  input_length=max_len,
                  mask_zero=False)(inp)
    x = Conv1D(filters=128, kernel_size=7, padding="same", activation="relu")(x)
    out = Conv1D(filters=num_classes, kernel_size=1, padding="same", activation="softmax")(x)
    model = keras.Model(inp, out)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# Afegeix la millor CNN al principi de la llista d'experiments
experiments = [
    ("conv_k7_nopool", build_conv_k7_nopool()),
    ("lstm_64",    build_rnn_token(cell="lstm", units=64,  bidirectional=False)),
    ("lstm_128",   build_rnn_token(cell="lstm", units=128, bidirectional=False)),
    ("bilstm_64",  build_rnn_token(cell="lstm", units=64,  bidirectional=True)),
    ("bilstm_128", build_rnn_token(cell="lstm", units=128, bidirectional=True)),
    ("gru_64",     build_rnn_token(cell="gru",  units=64,  bidirectional=False)),
    ("gru_128",    build_rnn_token(cell="gru",  units=128, bidirectional=False)),
    ("bigru_64",   build_rnn_token(cell="gru",  units=64,  bidirectional=True)),
    ("bigru_128",  build_rnn_token(cell="gru",  units=128, bidirectional=True)),
]

# ------------------------------------------------------------------
# Entrenament, avaluació i taula amb les mètriques finals
# ------------------------------------------------------------------
rows = []
for name, model in experiments:
    print("\n" + "="*90)
    print(f"Entrenando {name}  (embedding_dim=256)")
    print("="*90)

    model.fit(
        x=train_pad_sequences,
        y=train_labels_one_hot,
        sample_weight=train_w,
        validation_data=(val_pad_sequences, val_labels_one_hot, val_w),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )

    pre



Entrenando conv_k7_nopool  (embedding_dim=256)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.1751 - loss: 0.5585 - val_accuracy: 0.2191 - val_loss: 0.1396 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2297 - loss: 0.0859 - val_accuracy: 0.2294 - val_loss: 0.0911 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2383 - loss: 0.0387 - val_accuracy: 0.2322 - val_loss: 0.0809 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2416 - loss: 0.0229 - val_accuracy: 0.2334 - val_loss: 0.0785 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2431 - loss: 0.0159 - val_accuracy: 0.2339 - val_loss: 0.0784 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2437 - loss: 0.0120 - val_accuracy: 0.2342 - val_loss: 0.0794 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2444 - loss: 0.0091 -

 4. ** Transformer** Afegiu blocs de Transformer al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report

# =========================
# 0) Configuració
# =========================
tf.random.set_seed(7)
np.random.seed(7)

max_len     = int(train_pad_sequences.shape[1])
vocab_size  = int(np.max(train_pad_sequences)) + 1
num_classes = int(train_labels_one_hot.shape[-1])   # 120 (inclou <pad>)
embedding_dim = 256

# Pesos per ignorar <pad>
try:
    train_w, val_w, test_w
except NameError:
    PAD_IX = num_classes - 1
    train_w = (np.argmax(train_labels_one_hot, -1) != PAD_IX).astype("float32")
    val_w   = (np.argmax(val_labels_one_hot,   -1) != PAD_IX).astype("float32")
    test_w  = (np.argmax(test_labels_one_hot,  -1) != PAD_IX).astype("float32")

# Longituds reals
try:
    len_test_sequences
except NameError:
    len_test_sequences = np.sum(test_pad_sequences != 0, axis=1).tolist()

batch_size = 32
epochs = 10
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=2, factor=0.5),
]

# =========================
# Utilitats: mètriques
# =========================
def preds_to_index(preds, seq_lens):
    """Argmax per token tallant pel nombre real de tokens (ignora padding)."""
    idx = []
    for p, L in zip(preds, seq_lens):
        idx.extend(np.argmax(p[:L], axis=-1))
    return idx

def macro_table_row(name, y_true_oh, y_pred_proba, seq_lens):
    """Calcula valors agregats per a la taula (accuracy, macro i weighted)."""
    y_true_idx = preds_to_index(y_true_oh, seq_lens)
    y_pred_idx = preds_to_index(y_pred_proba, seq_lens)
    rep = classification_report(y_true_idx, y_pred_idx, output_dict=True, zero_division=0.0)
    return {
        "modelo": name,
        "acc":    rep.get("accuracy", 0.0),
        "prec_macro":  rep["macro avg"]["precision"],
        "rec_macro":   rep["macro avg"]["recall"],
        "f1_macro":    rep["macro avg"]["f1-score"],
        "prec_weight": rep["weighted avg"]["precision"],
        "rec_weight":  rep["weighted avg"]["recall"],
        "f1_weight":   rep["weighted avg"]["f1-score"],
    }

def print_results_table(rows):
    """Imprimeix la taula resum dels experiments."""
    header = f'{"modelo":20s}  acc    P_mac  R_mac  F1_mac  P_wgt  R_wgt  F1_wgt'
    print("\n" + header)
    print("-"*len(header))
    for r in rows:
        print(f'{r["modelo"]:20s}  {r["acc"]:.4f}  {r["prec_macro"]:.4f}  {r["rec_macro"]:.4f}  {r["f1_macro"]:.4f}  {r["prec_weight"]:.4f}  {r["rec_weight"]:.4f}  {r["f1_weight"]:.4f}')

# =========================
# Constructor Transformer (blocs propis)
# =========================
def build_transformer(name, num_heads=4, ff_dim=1024, depth=2, rate=0.1):
    """Token+Positional Embedding -> N blocs Transformer -> Dense per token."""
    inp = keras.Input(shape=(max_len,), dtype="int32")
    x = TokenAndPositionEmbedding(maxlen=max_len, vocab_size=vocab_size, embed_dim=embedding_dim)(inp)
    for _ in range(depth):
        x = TransformerBlock(embed_dim=embedding_dim, num_heads=num_heads, ff_dim=ff_dim, rate=rate)(x)
    out = Dense(num_classes, activation="softmax")(x)
    m = keras.Model(inp, out, name=name)
    m.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return m

# =========================
# Experiments (sense CNN/RNN)
# =========================
experiments = [
    ("trf_base_s_h4_d1_ff512",  build_transformer("trf_base_s_h4_d1_ff512",  num_heads=4, ff_dim=512,  depth=1, rate=0.1)),
    ("trf_base_m_h4_d2_ff1024", build_transformer("trf_base_m_h4_d2_ff1024", num_heads=4, ff_dim=1024, depth=2, rate=0.1)),
    ("trf_base_h_h8_d2_ff1024", build_transformer("trf_base_h_h8_d2_ff1024", num_heads=8, ff_dim=1024, depth=2, rate=0.1)),
    ("trf_base_d_h4_d3_ff1024", build_transformer("trf_base_d_h4_d3_ff1024", num_heads=4, ff_dim=1024, depth=3, rate=0.1)),
]

# =========================
# Entrenar + avaluar + taula
# =========================
rows = []
for name, model in experiments:
    print(f"\n=== Entrenando {name} ===")
    model.fit(
        train_pad_sequences, train_labels_one_hot,
        sample_weight=train_w,
        validation_data=(val_pad_sequences, val_labels_one_hot, val_w),
        epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=1
    )
    preds = model.predict(test_pad_sequences, batch_size=batch_size, verbose=0)
    rows.append(macro_table_row(name, test_labels_one_hot, preds, len_test_sequences))

print_results_table(rows)



=== Entrenando trf_base_s_h4_d1_ff512 ===
Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 14s 59ms/step - accuracy: 0.1946 - loss: 0.2379 - val_accuracy: 0.2139 - val_loss: 0.1140 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.2213 - loss: 0.0696 - val_accuracy: 0.2159 - val_loss: 0.1089 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2251 - loss: 0.0548 - val_accuracy: 0.2224 - val_loss: 0.0976 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2328 - loss: 0.0401 - val_accuracy: 0.2263 - val_loss: 0.0910 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2374 - loss: 0.0270 - val_accuracy: 0.2285 - val_loss: 0.0854 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2410 - loss: 0.0179 - val_accuracy: 0.2298 - val_loss: 0.0846 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━

##Regularització.
Quan proveu configuracions amb més paràmetres veureu que el model comença a tenir overfitting molt prompte durant l'entrenament. Afegiu Dropout al vostre model. Heu d'explicar la vostra decisió de valors i de posició dins de la xarxa.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, TimeDistributed, Dense, Dropout, SpatialDropout1D
from tensorflow.keras import Sequential
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

# Helpers (mètriques i conversió)
def preds_to_index(preds, seq_lens):
    """Argmax per token i tall per longitud real (ignora padding)."""
    idx_preds = []
    for pred, seq_len in zip(preds, seq_lens):
        for l in range(int(seq_len)):
            idx_preds.append(int(np.argmax(pred[l])))
    return idx_preds

def final_three_metrics(y_true_idx, y_pred_idx):
    """Retorna accuracy i macro/weighted (precision, recall, F1)."""
    acc = accuracy_score(y_true_idx, y_pred_idx)
    mP, mR, mF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="macro", zero_division=1.0
    )
    wP, wR, wF1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="weighted", zero_division=1.0
    )
    return {
        "accuracy": float(acc),
        "macro_precision": float(mP),
        "macro_recall": float(mR),
        "macro_f1": float(mF1),
        "weighted_precision": float(wP),
        "weighted_recall": float(wR),
        "weighted_f1": float(wF1),
        "support_total": int(len(y_true_idx)),
    }

# Longituds reals si no existeixen
try:
    len_test_sequences
except NameError:
    len_test_sequences = (test_pad_sequences != 0).sum(axis=1)

# Paràmetres fixos (LSTM_64)
embedding_dim = 256
vocab_size    = int(np.max(train_pad_sequences)) + 1
max_len       = int(train_pad_sequences.shape[1])
num_classes   = int(train_labels_one_hot.shape[-1])

batch_size = 32
epochs     = 10
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2),
]

# Builder LSTM64 (sense dropout intern)
def build_lstm64_dropout(emb_spatial_do=0.0,
                         post_lstm_do=0.0,
                         post_dense_do=0.0):
    """
    Embedding(mask_zero=True, dim=embedding_dim)
    -> [SpatialDropout1D opcional]
    -> LSTM(64, return_sequences=True)  # sense dropout intern
    -> [Dropout post-LSTM opcional]
    -> [TimeDistributed(Dense(128,relu)) + Dropout opcional]
    -> TimeDistributed(Dense(num_classes, softmax))
    """
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                        input_length=max_len, mask_zero=True))

    if emb_spatial_do and emb_spatial_do > 0:
        model.add(SpatialDropout1D(emb_spatial_do))

    model.add(LSTM(64, return_sequences=True))

    if post_lstm_do and post_lstm_do > 0:
        model.add(Dropout(post_lstm_do))

    if post_dense_do and post_dense_do > 0:
        model.add(TimeDistributed(Dense(128, activation="relu")))
        model.add(Dropout(post_dense_do))

    model.add(TimeDistributed(Dense(num_classes, activation="softmax")))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# Configuracions (on/off de drops externs)
dropout_levels = [0.05, 0.10, 0.20, 0.40, 0.60]

drop_configs = [
    ("all_on", dict(use_drop_embed=True, use_post_lstm=True, use_post_dense=True)),
    ("embed_postlstm", dict(use_drop_embed=True, use_post_lstm=True, use_post_dense=False)),
    ("embed_postdense", dict(use_drop_embed=True, use_post_lstm=False, use_post_dense=True)),
    ("embed_only", dict(use_drop_embed=True, use_post_lstm=False, use_post_dense=False)),
    ("postlstm_only", dict(use_drop_embed=False, use_post_lstm=True, use_post_dense=False)),
    ("postdense_only", dict(use_drop_embed=False, use_post_lstm=False, use_post_dense=True)),
]

# Entrenament + avaluació + taula
rows = []

for cfg_name, switches in drop_configs:
    for drop in dropout_levels:
        name = f"lstm_64_emb512__{cfg_name}__drop{drop}"
        print("\n" + "="*110)
        print(f"Entrenando {name}")
        print("="*110)

        emb_do  = drop if switches["use_drop_embed"] else 0.0
        post_do = drop if switches["use_post_lstm"]   else 0.0
        pdense  = drop if switches["use_post_dense"]  else 0.0

        model = build_lstm64_dropout(
            emb_spatial_do=emb_do,
            post_lstm_do=post_do,
            post_dense_do=pdense
        )

        model.fit(
            x=train_pad_sequences,
            y=train_labels_one_hot,
            sample_weight=train_w,
            validation_data=(val_pad_sequences, val_labels_one_hot, val_w),
            batch_size=batch_size,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1
        )

        preds = model.predict(test_pad_sequences, batch_size=batch_size, verbose=0)

        y_true_idx = preds_to_index(test_labels_one_hot, len_test_sequences)
        y_pred_idx = preds_to_index(preds,                len_test_sequences)

        met = final_three_metrics(y_true_idx, y_pred_idx)
        rows.append({
            "model": name,
            "accuracy": met["accuracy"],
            "macro_precision": met["macro_precision"],
            "macro_recall": met["macro_recall"],
            "macro_f1": met["macro_f1"],
            "weighted_precision": met["weighted_precision"],
            "weighted_recall": met["weighted_recall"],
            "weighted_f1": met["weighted_f1"],
            "support_total": met["support_total"],
        })

# Taula final
df = pd.DataFrame(rows)[[
    "model",
    "accuracy",
    "macro_precision", "macro_recall", "macro_f1",
    "weighted_precision", "weighted_recall", "weighted_f1",
    "support_total"
]].sort_values(["model"])

print("\nResumen LSTM64 (emb=512) SIN dropout interno:")
print(df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))



Entrenando lstm_64_emb512__all_on__drop0.05


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.1509 - loss: 2.9087 - val_accuracy: 0.1920 - val_loss: 0.9920 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2025 - loss: 0.7785 - val_accuracy: 0.2135 - val_loss: 0.6315 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.2231 - loss: 0.4166 - val_accuracy: 0.2246 - val_loss: 0.4637 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2330 - loss: 0.2444 - val_accuracy: 0.2293 - val_loss: 0.3979 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.2376 - loss: 0.1673 - val_accuracy: 0.2311 - val_loss: 0.3701 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2399 - loss: 0.1237 - val_accuracy: 0.2321 - val_loss: 0.3548 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2413 - loss: 0.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - accuracy: 0.1503 - loss: 2.9366 - val_accuracy: 0.1886 - val_loss: 0.9721 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2015 - loss: 0.7772 - val_accuracy: 0.2160 - val_loss: 0.6284 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2239 - loss: 0.4253 - val_accuracy: 0.2261 - val_loss: 0.4461 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2327 - loss: 0.2548 - val_accuracy: 0.2294 - val_loss: 0.3899 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.2372 - loss: 0.1751 - val_accuracy: 0.2313 - val_loss: 0.3629 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.2394 - loss: 0.1302 - val_accuracy: 0.2327 - val_loss: 0.3511 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2408 - loss: 0.

...

Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.1516 - loss: 2.8867 - val_accuracy: 0.1930 - val_loss: 0.9734 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2025 - loss: 0.7632 - val_accuracy: 0.2141 - val_loss: 0.6156 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.2228 - loss: 0.4124 - val_accuracy: 0.2267 - val_loss: 0.4449 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.2337 - loss: 0.2404 - val_accuracy: 0.2299 - val_loss: 0.3808 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2379 - loss: 0.1639 - val_accuracy: 0.2315 - val_loss: 0.3572 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2394 - loss: 0.1218 - val_accuracy: 0.2326 - val_loss: 0.3468 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2410 - loss: 0.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.1474 - loss: 2.9090 - val_accuracy: 0.1894 - val_loss: 1.0056 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2003 - loss: 0.8173 - val_accuracy: 0.2115 - val_loss: 0.6580 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2185 - loss: 0.4830 - val_accuracy: 0.2225 - val_loss: 0.4876 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.2294 - loss: 0.3051 - val_accuracy: 0.2279 - val_loss: 0.4098 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2349 - loss: 0.2130 - val_accuracy: 0.2304 - val_loss: 0.3770 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2375 - loss: 0.1658 - val_accuracy: 0.2319 - val_loss: 0.3603 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2396 - loss: 0.

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - accuracy: 0.1409 - loss: 3.0132 - val_accuracy: 0.1857 - val_loss: 1.0843 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.1902 - loss: 0.9347 - val_accuracy: 0.2076 - val_loss: 0.7139 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2143 - loss: 0.5703 - val_accuracy: 0.2207 - val_loss: 0.5369 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.2247 - loss: 0.3883 - val_accuracy: 0.2243 - val_loss: 0.4566 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.2300 - loss: 0.2908 - val_accuracy: 0.2284 - val_loss: 0.4099 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2336 - loss: 0.2299 - val_accuracy: 0.2303 - val_loss: 0.3809 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2360 - loss: 0

##DESBALANCEIG DE CLASSES

In [ ]:
# Nombres de clase
try:
    label_names = list(label_encoder.classes_) + ['<pad>']
except NameError:
    label_names = [f"cls_{i}" for i in range(num_classes-1)] + ['<pad>']

import pandas as pd

def freq_table(counts_vec, label_names):
    df = pd.DataFrame({
        "class_id": np.arange(len(counts_vec)),
        "class_name": label_names,
        "count": counts_vec
    })
    df["pct"] = df["count"] / df["count"].sum()
    return df.sort_values("count", ascending=False)

df_freq = freq_table(counts_all, label_names)
print(df_freq.head(20).to_string(index=False))


 class_id                  class_name  count      pct
      119                       <pad> 204065 0.757030
      118                           O  41449 0.153765
       72           B-toloc.city_name   5036 0.018682
       45         B-fromloc.city_name   5019 0.018619
      115           I-toloc.city_name   1359 0.005042
       24      B-depart_date.day_name   1094 0.004058
      103         I-fromloc.city_name    867 0.003216
        2              B-airline_name    800 0.002968
       31 B-depart_time.period_of_day    716 0.002656
       77              I-airline_name    489 0.001814
       25    B-depart_date.day_number    448 0.001662
       26    B-depart_date.month_name    435 0.001614
       33          B-depart_time.time    423 0.001569
       61                B-round_trip    418 0.001551
      110                I-round_trip    406 0.001506
       34 B-depart_time.time_relative    388 0.001439
       19             B-cost_relative    381 0.001413
       39                B-f

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

# Configuració bàsica
tf.random.set_seed(7)
np.random.seed(7)

max_len     = int(train_pad_sequences.shape[1])
vocab_size  = int(np.max(train_pad_sequences)) + 1
num_classes = int(train_labels_one_hot.shape[-1])   # inclou <pad>
embedding_dim = 256

pad_token_id_inputs = 0
pad_label_id        = num_classes - 1

try:
    len_test_sequences
except NameError:
    len_test_sequences = np.sum(test_pad_sequences != pad_token_id_inputs, axis=1).tolist()

batch_size = 32
epochs = 10
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=2, factor=0.5),
]

# Funcions de mètriques i resum
def preds_to_index(preds_or_oh, seq_lens):
    idx = []
    for p, L in zip(preds_or_oh, seq_lens):
        idx.extend(np.argmax(p[:int(L)], axis=-1))
    return idx

def macro_table_row(name, y_true_oh, y_pred_proba, seq_lens):
    """Crea una fila amb les mètriques macro i ponderades."""
    y_true_idx = preds_to_index(y_true_oh, seq_lens)
    y_pred_idx = preds_to_index(y_pred_proba, seq_lens)
    rep = classification_report(y_true_idx, y_pred_idx, output_dict=True, zero_division=0.0)
    return {
        "modelo": name,
        "acc": rep.get("accuracy", 0.0),
        "prec_macro": rep["macro avg"]["precision"],
        "rec_macro": rep["macro avg"]["recall"],
        "f1_macro": rep["macro avg"]["f1-score"],
        "prec_weight": rep["weighted avg"]["precision"],
        "rec_weight": rep["weighted avg"]["recall"],
        "f1_weight": rep["weighted avg"]["f1-score"],
    }

def print_results_table(rows):
    """Mostra la taula resum dels resultats."""
    header = f'{"modelo":28s}  acc    P_mac  R_mac  F1_mac  P_wgt  R_wgt  F1_wgt'
    print("\n" + header)
    print("-" * len(header))
    for r in rows:
        print(f'{r["modelo"]:28s}  {r["acc"]:.4f}  {r["prec_macro"]:.4f}  {r["rec_macro"]:.4f}  {r["f1_macro"]:.4f}  {r["prec_weight"]:.4f}  {r["rec_weight"]:.4f}  {r["f1_weight"]:.4f}')

# Ponderació de classes i creació de pesos per token
def build_token_weights(x_pad, y_ids, cw_vec, pad_id=0):
    """Genera pesos per token segons la seva classe (0 per <pad>)."""
    mask_no_pad = (x_pad != pad_id).astype(np.float32)
    return mask_no_pad * cw_vec[y_ids]

train_y_ids = np.argmax(train_labels_one_hot, axis=-1)
val_y_ids   = np.argmax(val_labels_one_hot, axis=-1)
test_y_ids  = np.argmax(test_labels_one_hot, axis=-1)

mask_train_flat = (train_pad_sequences.flatten() != pad_token_id_inputs)
valid_flat_y = train_y_ids.flatten()[mask_train_flat]

present_classes = np.unique(valid_flat_y)
cw_present = compute_class_weight('balanced', classes=present_classes, y=valid_flat_y).astype(np.float32)

cw_full = np.ones(num_classes, dtype=np.float32)
cw_full[present_classes] = cw_present
cw_full[present_classes] = np.sqrt(cw_full[present_classes])
cw_full[present_classes] = np.clip(cw_full[present_classes], 0.5, 5.0)
cw_full[pad_label_id] = 0.0

train_sw = build_token_weights(train_pad_sequences, train_y_ids, cw_full, pad_id=pad_token_id_inputs)
val_sw   = build_token_weights(val_pad_sequences,   val_y_ids,   cw_full, pad_id=pad_token_id_inputs)
test_sw  = build_token_weights(test_pad_sequences,  test_y_ids,  cw_full, pad_id=pad_token_id_inputs)

print(f"[train_sw] min:{train_sw.min():.4f} max:{train_sw.max():.4f} mean:{train_sw.mean():.4f}")
print(f"[val_sw]   min:{val_sw.min():.4f} max:{val_sw.max():.4f} mean:{val_sw.mean():.4f}")
print(f"[test_sw]  min:{test_sw.min():.4f} max:{test_sw.max():.4f} mean:{test_sw.mean():.4f}")

# Model Transformer
def build_transformer(name, num_heads=4, ff_dim=1024, depth=2, rate=0.1):
    """Construeix un model Transformer senzill per seqüències etiquetades."""
    inp = keras.Input(shape=(max_len,), dtype="int32")
    x = TokenAndPositionEmbedding(maxlen=max_len, vocab_size=vocab_size, embed_dim=embedding_dim)(inp)
    for _ in range(depth):
        x = TransformerBlock(embed_dim=embedding_dim, num_heads=num_heads, ff_dim=ff_dim, rate=rate)(x)
    out = Dense(num_classes, activation="softmax")(x)
    m = keras.Model(inp, out, name=name)
    m.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return m

# Entrenament i resultats
experiments = [
    ("trf_base_m_h4_d2_ff1024", build_transformer("trf_base_m_h4_d2_ff1024", num_heads=4, ff_dim=1024, depth=2, rate=0.1)),
    ("trf_base_h_h8_d2_ff1024", build_transformer("trf_base_h_h8_d2_ff1024", num_heads=8, ff_dim=1024, depth=2, rate=0.1)),
]

rows = []
for name, model in experiments:
    print(f"\n=== Entrenant {name} (emb=512, amb pesos per token) ===")
    model.fit(
        train_pad_sequences, train_labels_one_hot,
        sample_weight=train_sw,
        validation_data=(val_pad_sequences, val_labels_one_hot, val_sw),
        epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=1
    )
    _ = model.evaluate(test_pad_sequences, test_labels_one_hot, sample_weight=test_sw, verbose=0)
    preds = model.predict(test_pad_sequences, batch_size=batch_size, verbose=0)
    rows.append(macro_table_row(name, test_labels_one_hot, preds, len_test_sequences))

print_results_table(rows)


[train_sw] min:0.0000 max:5.0000 mean:0.1772
[val_sw]   min:0.0000   max:5.0000   mean:0.1740
[test_sw]  min:0.0000  max:5.0000  mean:0.1656

=== Entrenando trf_base_m_h4_d2_ff1024 (emb=512, token sample_weight) ===
Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 29s 88ms/step - accuracy: 0.1859 - loss: 0.3396 - val_accuracy: 0.2148 - val_loss: 0.1397 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.2252 - loss: 0.0775 - val_accuracy: 0.2244 - val_loss: 0.1204 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.2343 - loss: 0.0472 - val_accuracy: 0.2232 - val_loss: 0.1182 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.2355 - loss: 0.0401 - val_accuracy: 0.2286 - val_loss: 0.1058 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.2386 - loss: 0.0315 - val_accuracy: 0.2302 - val_loss: 0.1064 - learning_rate: 0.0010
Epoch 6/10
128/128

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, TimeDistributed, Dense
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import pandas as pd

# =========================
# 0) Setup bàsic
# =========================
tf.random.set_seed(7)
np.random.seed(7)

max_len     = int(train_pad_sequences.shape[1])
vocab_size  = int(np.max(train_pad_sequences)) + 1
num_classes = int(train_labels_one_hot.shape[-1])
embedding_dim = 256

pad_token_id_inputs = 0             # padding a les seqüències d'entrada
pad_label_id        = num_classes-1 # <pad> és la darrera classe al one-hot

try:
    len_test_sequences
except NameError:
    len_test_sequences = np.sum(test_pad_sequences != pad_token_id_inputs, axis=1).tolist()

batch_size = 32
epochs = 10
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=2, factor=0.5),
]

# =========================
# 1) Utilitats
# =========================
def preds_to_index(preds_or_oh, seq_lens):
    idx = []
    for p, L in zip(preds_or_oh, seq_lens):
        idx.extend(np.argmax(p[:int(L)], axis=-1))
    return idx

def macro_table_row(name, y_true_oh, y_pred_proba, seq_lens):
    y_true_idx = preds_to_index(y_true_oh,  seq_lens)
    y_pred_idx = preds_to_index(y_pred_proba, seq_lens)
    rep = classification_report(y_true_idx, y_pred_idx, output_dict=True, zero_division=0.0)
    return {
        "modelo": name,
        "acc":    rep.get("accuracy", 0.0),
        "prec_macro":  rep["macro avg"]["precision"],
        "rec_macro":   rep["macro avg"]["recall"],
        "f1_macro":    rep["macro avg"]["f1-score"],
        "prec_weight": rep["weighted avg"]["precision"],
        "rec_weight":  rep["weighted avg"]["recall"],
        "f1_weight":   rep["weighted avg"]["f1-score"],
    }

def print_results_table(rows):
    header = f'{"modelo":28s}  acc    P_mac  R_mac  F1_mac  P_wgt  R_wgt  F1_wgt'
    print("\n" + header)
    print("-"*len(header))
    for r in rows:
        print(f'{r["modelo"]:28s}  {r["acc"]:.4f}  {r["prec_macro"]:.4f}  {r["rec_macro"]:.4f}  {r["f1_macro"]:.4f}  {r["prec_weight"]:.4f}  {r["rec_weight"]:.4f}  {r["f1_weight"]:.4f}')

# =========================
# 2) Pesos per classe (NOMÉS TRAIN) -> sample_weight per token
# =========================
def build_token_weights(x_pad, y_ids, cw_vec, pad_id=0):
    mask_no_pad = (x_pad != pad_id).astype(np.float32)
    return mask_no_pad * cw_vec[y_ids]

train_y_ids = np.argmax(train_labels_one_hot, axis=-1)
val_y_ids   = np.argmax(val_labels_one_hot,   axis=-1)
test_y_ids  = np.argmax(test_labels_one_hot,  axis=-1)

mask_train_flat = (train_pad_sequences.flatten() != pad_token_id_inputs)
valid_flat_y    = train_y_ids.flatten()[mask_train_flat]

present_classes = np.unique(valid_flat_y)
cw_present = compute_class_weight(class_weight='balanced', classes=present_classes, y=valid_flat_y).astype(np.float32)

cw_full = np.ones(num_classes, dtype=np.float32)
cw_full[present_classes] = cw_present
cw_full[present_classes] = np.sqrt(cw_full[present_classes])              # suavitza extrems
cw_full[present_classes] = np.clip(cw_full[present_classes], 0.5, 5.0)    # evita pesos desbocats
cw_full[pad_label_id] = 0.0                                               # <pad> no compta

train_sw = build_token_weights(train_pad_sequences, train_y_ids, cw_full, pad_id=pad_token_id_inputs)
val_sw   = build_token_weights(val_pad_sequences,   val_y_ids,   cw_full, pad_id=pad_token_id_inputs)
test_sw  = build_token_weights(test_pad_sequences,  test_y_ids,  cw_full, pad_id=pad_token_id_inputs)

# =========================
# 3) Constructors BiLSTM128 i BiGRU128 per token
# =========================
def build_bilstm128(name="bilstm_128_emb256"):
    inp = keras.Input(shape=(max_len,), dtype="int32")
    x = Embedding(vocab_size, embedding_dim, input_length=max_len, mask_zero=True)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    out = TimeDistributed(Dense(num_classes, activation="softmax"))(x)
    m = keras.Model(inp, out, name=name)
    m.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return m

def build_bigru128(name="bigru_128_emb256"):
    inp = keras.Input(shape=(max_len,), dtype="int32")
    x = Embedding(vocab_size, embedding_dim, input_length=max_len, mask_zero=True)(inp)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    out = TimeDistributed(Dense(num_classes, activation="softmax"))(x)
    m = keras.Model(inp, out, name=name)
    m.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return m

# =========================
# 4) Entrenament + Avaluació amb sample_weight per a ambdós models
# =========================
rows = []
for name, builder in [
    ("bilstm_128_emb256+weights", build_bilstm128),
    ("bigru_128_emb256+weights",  build_bigru128),
]:
    print(f"\n=== Entrenant {name} ===")
    model = builder()
    model.fit(
        train_pad_sequences, train_labels_one_hot,
        sample_weight=train_sw,
        validation_data=(val_pad_sequences, val_labels_one_hot, val_sw),
        epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=1
    )
    _ = model.evaluate(test_pad_sequences, test_labels_one_hot, sample_weight=test_sw, verbose=0)
    preds = model.predict(test_pad_sequences, batch_size=batch_size, verbose=0)
    rows.append(macro_table_row(name, test_labels_one_hot, preds, len_test_sequences))

print_results_table(rows)



=== Entrenant bilstm_128_emb256+weights ===


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.1537 - loss: 2.3301 - val_accuracy: 0.2027 - val_loss: 1.2578 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2175 - loss: 0.9879 - val_accuracy: 0.2270 - val_loss: 0.6391 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2358 - loss: 0.4329 - val_accuracy: 0.2327 - val_loss: 0.4184 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2411 - loss: 0.2159 - val_accuracy: 0.2341 - val_loss: 0.3301 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2434 - loss: 0.1240 - val_accuracy: 0.2347 - val_loss: 0.3030 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.2443 - loss: 0.0812 - val_accuracy: 0.2353 - val_loss: 0.2766 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2450 - loss: 0

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.1623 - loss: 2.1905 - val_accuracy: 0.2209 - val_loss: 0.8758 - learning_rate: 0.0010
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2313 - loss: 0.6072 - val_accuracy: 0.2306 - val_loss: 0.4593 - learning_rate: 0.0010
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2388 - loss: 0.2604 - val_accuracy: 0.2328 - val_loss: 0.3619 - learning_rate: 0.0010
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2418 - loss: 0.1532 - val_accuracy: 0.2339 - val_loss: 0.3235 - learning_rate: 0.0010
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2434 - loss: 0.0975 - val_accuracy: 0.2344 - val_loss: 0.3055 - learning_rate: 0.0010
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2444 - loss: 0.0604 - val_accuracy: 0.2351 - val_loss: 0.2956 - learning_rate: 0.0010
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2452 - loss: 0.

La bilstm128 ha millorat respecte al seu baseline

---

<h1><a name="section-four"> 4. Lliurable </a></h1>

Heu d'entregar un document PDF de com a **màxim 10 pàgines** que incloga els resultats de tots els exercicis així com una explicació de cadascun dels resultats i de la modificació que heu fet. L'estructura del document és:

1. Introducció.
2. Experiments i Resultats (amb raonament).
3. Conclusions.

No cal que afegiu el vostre codi al document, podeu entregar el *notebook* juntament amb el document.

 ---